In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
TRAIN_DIR = '../dataset/train'
TEST_DIR = '../dataset/test'

In [3]:
import os

In [4]:
MAP_CHARACTERS = {}

for idx, training_class in enumerate(os.listdir(TRAIN_DIR)):
    MAP_CHARACTERS[idx] = training_class

In [5]:
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=.1)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='training'
)

val_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='validation'
)

Found 14860 images belonging to 32 classes.
Found 1636 images belonging to 32 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(
    directory=TEST_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True
)

Found 4142 images belonging to 32 classes.


In [8]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [9]:
tf.keras.backend.clear_session()

In [10]:
resnet = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet')
resnet.trainable = False

# tf.keras.applications.EfficientNetB5(
# include_top=True, weights='imagenet'

In [11]:
inputs = tf.keras.layers.Input((224, 224, 3))

In [12]:
x = resnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

In [13]:
outputs = tf.keras.layers.Dense(32, activation='softmax')(x)

In [14]:
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
Total params: 23,698,016
Trainable params: 133,216
Non-trainable params: 23,564,800
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit(train_generator,
          validation_data=val_generator,
          epochs=10)

Epoch 1/10
465/465 [==============================] - 455s 975ms/step - loss: 2.3666 - accuracy: 0.3615 - val_loss: 1.6992 - val_accuracy: 0.5465
Epoch 2/10
465/465 [==============================] - 449s 966ms/step - loss: 1.1672 - accuracy: 0.6943 - val_loss: 1.5322 - val_accuracy: 0.5880
Epoch 3/10
465/465 [==============================] - 438s 942ms/step - loss: 0.8803 - accuracy: 0.7641 - val_loss: 1.4035 - val_accuracy: 0.6229
Epoch 4/10
465/465 [==============================] - 436s 937ms/step - loss: 0.6815 - accuracy: 0.8166 - val_loss: 1.3925 - val_accuracy: 0.6351
Epoch 5/10
465/465 [==============================] - 454s 976ms/step - loss: 0.5617 - accuracy: 0.8506 - val_loss: 1.4414 - val_accuracy: 0.6363
Epoch 6/10
465/465 [==============================] - 453s 975ms/step - loss: 0.4621 - accuracy: 0.8750 - val_loss: 1.4987 - val_accuracy: 0.6302
Epoch 7/10
465/465 [==============================] - 436s 938ms/step - loss: 0.3810 - accuracy: 0.9003 - val_loss: 1.5242 -